# Liệu những champion nào sẽ được dùng cùng nhau nhiều nhất trong các trận đấu của những người chơi? Câu hỏi tương tự với các trang bị?

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
from apyori import apriori
from sklearn.datasets import load_iris
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder


### Đọc data

In [2]:
# To make sure the first row is not thought of as the heading
dataset = pd.read_csv('../Data/process_data_tft_each_item_csv.csv', usecols=[
                      'ID Match', 'List Name Item', 'Champion']).drop_duplicates().reset_index(drop=True)


## Sử dụng thuật khai thác mẫu phổ biến với cây FP để tìm ra các tập hợp có support nhiều nhất tương ứng với sử dụng nhiều nhất (Apriori không được sử dụng vì số transaction rất nhiều nên sẽ bị time out)

### Với các Champion 
Đầu tiên tạo mảng 2 chiều lưu danh sách các đội hình chứa các champion của mõi người chơi

In [3]:
list_items = []
for i in range(len(dataset['Champion'])):
    list_items.append(ast.literal_eval(dataset['Champion'][i]))
list_items


[['TFT8_Gangplank',
  'TFT8_Vi',
  'TFT8_LeeSin',
  'TFT8_Malphite',
  'TFT8_Ezreal',
  'TFT8_Sona',
  'TFT8_Kaisa'],
 ['TFT8_Lux',
  'TFT8_Annie',
  'TFT8_Yuumi',
  'TFT8_Sona',
  'TFT8_Alistar',
  'TFT8_Ekko',
  'TFT8_Taliyah',
  'TFT8_Syndra'],
 ['TFT8_Galio',
  'TFT8_Nasus',
  'TFT8_Gangplank',
  'TFT8_LeeSin',
  'TFT8_Malphite',
  'TFT8_Yuumi',
  'TFT8_Alistar',
  'TFT8_Nunu'],
 ['TFT8_Fiora',
  'TFT8_Yasuo',
  'TFT8_Vayne',
  'TFT8_Nilah',
  'TFT8_Zoe',
  'TFT8_Leblanc',
  'TFT8_Zed',
  'TFT8_Sejuani'],
 ['TFT8_Lulu',
  'TFT8_Annie',
  'TFT8_Yuumi',
  'TFT8_Sona',
  'TFT8_Alistar',
  'TFT8_Zoe',
  'TFT8_Ekko',
  'TFT8_Soraka'],
 ['TFT8_Talon',
  'TFT8_Blitzcrank',
  'TFT8_Vi',
  'TFT8_Camille',
  'TFT8_Zoe',
  'TFT8_Leblanc',
  'TFT8_Viego',
  'TFT8_Soraka'],
 ['TFT8_Sivir',
  'TFT8_Velkoz',
  'TFT8_Chogath',
  'TFT8_Rammus',
  'TFT8_Rammus',
  'TFT8_Samira',
  'TFT8_AurelionSol',
  'TFT8_Urgot'],
 ['TFT8_Lulu',
  'TFT8_Annie',
  'TFT8_Yuumi',
  'TFT8_Alistar',
  'TFT8_Zoe',
  'T

Sử dụng apply valuecounts để tạo dataframe 2 chiều với index là tất cả trận đấu còn column là tên các champion của trò chơi, mỗi hàng là 1 transaction, vì đôi lúc người chơi sẽ dùng 2 champion trùng nhau trong 1 trận đấu nên thay thế các số 2, 3, 4 thành 1 để FP tree hoạt động được

In [4]:
df_champion = pd.DataFrame(list_items).apply(pd.value_counts, 1).fillna(0)

In [5]:
df_champion = df_champion.replace(2, 1)
df_champion = df_champion.replace(3, 1)
df_champion = df_champion.replace(4, 1)


In [6]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df_champion[:5])


   TFT8_Alistar  TFT8_Annie  TFT8_Aphelios  TFT8_Ashe  TFT8_AurelionSol  \
0           0.0         0.0            0.0        0.0               0.0   
1           1.0         1.0            0.0        0.0               0.0   
2           1.0         0.0            0.0        0.0               0.0   
3           0.0         0.0            0.0        0.0               0.0   
4           1.0         1.0            0.0        0.0               0.0   

   TFT8_BelVeth  TFT8_Blitzcrank  TFT8_Camille  TFT8_Chogath  TFT8_Draven  \
0           0.0              0.0           0.0           0.0          0.0   
1           0.0              0.0           0.0           0.0          0.0   
2           0.0              0.0           0.0           0.0          0.0   
3           0.0              0.0           0.0           0.0          0.0   
4           0.0              0.0           0.0           0.0          0.0   

   TFT8_Ekko  TFT8_Ezreal  TFT8_Fiddlesticks  TFT8_Fiora  TFT8_Galio  \
0        0.0  

**Áp cây fp để sinh tập phổ biến với min_sup = 0.1, sort theo support**

In [7]:
res=fpgrowth(df_champion,min_support=0.1, use_colnames=True)
res = res.sort_values('support', ascending=False)


c:\Users\ACER\anaconda3\envs\min_ds-env\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [8]:
res.head()

,support,itemsets
6,0.466015,(TFT8_Alistar)
7,0.450265,(TFT8_Ekko)
0,0.341291,(TFT8_Vi)
14,0.338417,(TFT8_Sejuani)
43,0.295590,"(TFT8_Alistar, TFT8_Ekko)"


**Xuất các bộ k champ đi cùng nhau**

3 champion hay đi cùng nhau

In [9]:
res.loc[res["itemsets"].apply(lambda x: len(x) == 3)]


,support,itemsets
40,0.152796,"(TFT8_Gangplank, TFT8_Malphite, TFT8_LeeSin)"
48,0.132681,"(TFT8_Annie, TFT8_Alistar, TFT8_Ekko)"
76,0.107538,"(TFT8_Sejuani, TFT8_Jax, TFT8_Vi)"
71,0.105603,"(TFT8_Riven, TFT8_Sejuani, TFT8_Vi)"
77,0.103172,"(TFT8_Riven, TFT8_Sejuani, TFT8_Jax)"
78,0.102619,"(TFT8_Riven, TFT8_Jax, TFT8_Vi)"


2 champion hay đi cùng nhau

In [10]:
res.loc[res["itemsets"].apply(lambda x: len(x) == 2)]


,support,itemsets
43,0.295590,"(TFT8_Alistar, TFT8_Ekko)"
54,0.213749,"(TFT8_Sejuani, TFT8_Vi)"
44,0.165064,"(TFT8_Annie, TFT8_Alistar)"
37,0.158267,"(TFT8_Malphite, TFT8_LeeSin)"
38,0.154178,"(TFT8_Gangplank, TFT8_Malphite)"
36,0.153957,"(TFT8_Gangplank, TFT8_LeeSin)"
32,0.153570,"(TFT8_Ekko, TFT8_Vi)"
57,0.152188,"(TFT8_Zoe, TFT8_Ekko)"
45,0.149646,"(TFT8_Annie, TFT8_Ekko)"
69,0.147270,"(TFT8_Riven, TFT8_Vi)"


**Rút ra luật kết hợp với metric = lift, lift càng cao thì độ phụ thuộc vào nhau của vế trái phải càng lớn**

In [11]:
temp=association_rules(res, metric="lift", min_threshold=1)

In [12]:
temp.sort_values('lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
13,"(TFT8_Gangplank, TFT8_LeeSin)",(TFT8_Malphite),0.153957,0.169595,0.152796,0.992462,5.851938,0.126686,110.166998
16,(TFT8_Malphite),"(TFT8_Gangplank, TFT8_LeeSin)",0.169595,0.153957,0.152796,0.900945,5.851938,0.126686,8.541141
15,(TFT8_Gangplank),"(TFT8_Malphite, TFT8_LeeSin)",0.175785,0.158267,0.152796,0.869224,5.492133,0.124975,6.436425
14,"(TFT8_Malphite, TFT8_LeeSin)",(TFT8_Gangplank),0.158267,0.175785,0.152796,0.965433,5.492133,0.124975,23.843966
8,(TFT8_Gangplank),(TFT8_Malphite),0.175785,0.169595,0.154178,0.877083,5.171616,0.124365,6.755797
...,...,...,...,...,...,...,...,...,...
54,(TFT8_Sejuani),(TFT8_LeeSin),0.338417,0.282770,0.115164,0.340300,1.203455,0.019469,1.087208
49,(TFT8_Alistar),(TFT8_Leblanc),0.466015,0.211594,0.117816,0.252816,1.194820,0.019210,1.055171
48,(TFT8_Leblanc),(TFT8_Alistar),0.211594,0.466015,0.117816,0.556803,1.194820,0.019210,1.204850
44,(TFT8_Alistar),(TFT8_Zoe),0.466015,0.223198,0.120192,0.257915,1.155542,0.016179,1.046783


### TƯƠNG TỰ CHAMPION, TA CÓ TRANG BỊ

In [13]:
list_items = []
for i in range(len(dataset['List Name Item'])):
    list_items.append(ast.literal_eval(dataset['List Name Item'][i]))
list_items = [elem for twod in list_items for elem in twod]
list_items = [elem for elem in list_items if elem != []]
list_items


[['TFT_Item_ZekesHerald'],
 ['TFT_Item_BrambleVest', 'TFT_Item_TitansResolve', 'TFT_Item_DragonsClaw'],
 ['TFT_Item_DragonsClaw', 'TFT_Item_Bloodthirster'],
 ['TFT_Item_SpearOfShojin',
  'TFT_Item_ArchangelsStaff',
  'TFT_Item_HextechGunblade'],
 ['TFT_Item_SeraphsEmbrace',
  'TFT_Item_ArchangelsStaff',
  'TFT_Item_InfinityEdge'],
 ['TFT8_Item_HeartEmblemItem'],
 ['TFT_Item_BrambleVest',
  'TFT_Item_GargoyleStoneplate',
  'TFT_Item_Quicksilver'],
 ['TFT_Item_SeraphsEmbrace'],
 ['TFT_Item_GiantsBelt'],
 ['TFT_Item_JeweledGauntlet',
  'TFT_Item_SpearOfShojin',
  'TFT_Item_PowerGauntlet'],
 ['TFT_Item_GargoyleStoneplate'],
 ['TFT_Item_UnstableConcoction',
  'TFT_Item_JeweledGauntlet',
  'TFT_Item_GuardianAngel'],
 ['TFT_Item_MadredsBloodrazor',
  'TFT_Item_JeweledGauntlet',
  'TFT_Item_SeraphsEmbrace'],
 ['TFT_Item_Morellonomicon'],
 ['TFT8_Item_HackerEmblemItem'],
 ['TFT_Item_Shroud'],
 ['TFT_Item_Zephyr'],
 ['TFT8_Item_DuelistEmblemItem'],
 ['TFT_Item_GuardianAngel', 'TFT_Item_LastWhisp

In [14]:
te = TransactionEncoder()
te_ary = te.fit(list_items).transform(list_items)
df_itemset = pd.DataFrame(te_ary, columns=te.columns_)


# temp = df_itemset.copy()


In [15]:
frequent_itemsets = fpgrowth(df_itemset, min_support=0.001, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values('support', ascending=False)


In [16]:
frequent_itemsets.head()

,support,itemsets
18,0.123983,(TFT_Item_MadredsBloodrazor)
14,0.107192,(TFT_Item_JeweledGauntlet)
36,0.099222,(TFT_Item_RedBuff)
42,0.081129,(TFT_Item_IonicSpark)
28,0.079633,(TFT_Item_WarmogsArmor)


In [17]:
item_k_3 = frequent_itemsets.loc[frequent_itemsets["itemsets"].apply(
    lambda x: len(x) == 3)]
print(item_k_3[:10].to_string())


      support                                                                          itemsets
190  0.010807   (TFT_Item_JeweledGauntlet, TFT_Item_SeraphsEmbrace, TFT_Item_MadredsBloodrazor)
147  0.008344    (TFT_Item_MadredsBloodrazor, TFT_Item_SpearOfShojin, TFT_Item_JeweledGauntlet)
232  0.004488        (TFT_Item_GuinsoosRageblade, TFT_Item_Quicksilver, TFT_Item_Bloodthirster)
98   0.003430      (TFT_Item_GuinsoosRageblade, TFT_Item_TitansResolve, TFT_Item_Bloodthirster)
202  0.003327           (TFT_Item_TitansResolve, TFT_Item_InfinityEdge, TFT_Item_Bloodthirster)
165  0.003314     (TFT_Item_JeweledGauntlet, TFT_Item_SeraphsEmbrace, TFT_Item_HextechGunblade)
332  0.003185                      (TFT_Item_Redemption, TFT_Item_RedBuff, TFT_Item_IonicSpark)
346  0.002992                    (TFT_Item_WarmogsArmor, TFT_Item_RedBuff, TFT_Item_IonicSpark)
417  0.002876         (TFT_Item_LastWhisper, TFT_Item_RunaansHurricane, TFT_Item_GuardianAngel)
359  0.002721  (TFT_Item_SeraphsEmbrace,

In [18]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(TFT_Item_JeweledGauntlet),(TFT_Item_MadredsBloodrazor),0.107192,0.123983,0.032395,0.302214,2.437545,0.019105,1.255423
1,(TFT_Item_MadredsBloodrazor),(TFT_Item_JeweledGauntlet),0.123983,0.107192,0.032395,0.261286,2.437545,0.019105,1.208597
2,(TFT_Item_JeweledGauntlet),(TFT_Item_SpearOfShojin),0.107192,0.072012,0.027856,0.259865,3.608655,0.020136,1.253810
3,(TFT_Item_SpearOfShojin),(TFT_Item_JeweledGauntlet),0.072012,0.107192,0.027856,0.386819,3.608655,0.020136,1.456028
4,(TFT_Item_JeweledGauntlet),(TFT_Item_SeraphsEmbrace),0.107192,0.077983,0.027791,0.259264,3.324638,0.019432,1.244731
...,...,...,...,...,...,...,...,...,...
817,(TFT_Item_GuinsoosRageblade),"(TFT_Item_StatikkShiv, TFT_Item_MadredsBloodra...",0.073598,0.005868,0.001019,0.013843,2.359122,0.000587,1.008087
818,(TFT_Item_MadredsBloodrazor),"(TFT_Item_GuinsoosRageblade, TFT_Item_StatikkS...",0.123983,0.006035,0.001019,0.008217,1.361507,0.000271,1.002200
819,(TFT_Item_StatikkShiv),"(TFT_Item_GuinsoosRageblade, TFT_Item_MadredsB...",0.047922,0.016185,0.001019,0.021259,1.313561,0.000243,1.005185
820,(TFT_Item_RedBuff),(TFT4_Item_OrnnAnimaVisage),0.099222,0.003611,0.001006,0.010138,2.807547,0.000648,1.006594
